In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

3.4917022776198507 108.0741059830309
60.43685581311095 147.9447157251608
-85.75603112771168 169.3310241142825
-0.6606321172503158 -4.495735040308205
-66.61244998168705 -167.04288442780674
65.09754218725311 -163.43534130085098
-85.18822088326618 -0.8652409880569394
-9.478402903056377 -104.35849066235457
-66.18194266638984 79.07444997062544
-51.293189525018796 103.63199125386274
-63.475301342650326 -107.31530330268274
55.92246287174413 65.5572491384006
25.98036742643849 63.91114117576339
-73.48002773787151 44.465165783360135
-3.4047114682334865 -141.72448918590936
12.48637923626002 137.0990881855697
73.62184366694277 -49.97146910363023
75.12202688341611 47.13075013441028
-22.76064418817147 -170.3609708222771
-21.767882619144245 123.56370123499318
-17.39128746503812 -92.6030997784114
-20.834429207076695 129.67649506407804
4.604452876661156 -42.82054598698713
-73.64759821450181 -171.9266983519053
-55.68048160223855 -157.07771491976118
46.78749684688796 145.6011396343394
-83.54535216999534 

-68.9481363851711 102.51895119113834
-68.86436348583919 -20.40098489249735
-57.389863617768604 153.17277497007774
-29.54417842747999 -149.14163276216266
78.3079319132581 -96.09805857682869
39.12242339266143 149.08706042748605
47.08717533935683 99.10073077067153
-66.90250329683798 -117.36620017955208
-82.08151167462701 11.028447182523507
-56.44628406052147 6.4985926712136575
50.0842590715944 168.68740567457962
-76.45560522997636 44.57899189341208
-73.09909001274802 166.43666296069085
42.11148056965317 148.2067329957764
-54.738222057836424 39.225481680409814
-15.449096698512406 -156.08193892254508
-2.9072337576948826 -36.35985330010561
-89.40049597414077 63.882052664245094
-32.81505584550337 -103.74522918378507
54.4003946077776 -117.46867193659989
-20.876144749691747 -21.040509284491804
-28.787285860682424 129.37775727055043
-16.810145954647822 54.9805909717297
49.005795664246904 -120.82211429776837
-21.791383018241362 -104.11348198373089
40.32039972451744 57.385147467681435
31.754169323

In [5]:
from citipy import citipy

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

589

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=bea5d4a9a7b7fef5b2d61a5aeef034c6


In [8]:
# Make a 'Get' request for the city weather.
city_weather = requests.get(city_url)
city_weather

NameError: name 'city_url' is not defined

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | pemangkat
Processing Record 2 of Set 1 | ust-omchug
Processing Record 3 of Set 1 | bluff
Processing Record 4 of Set 1 | san-pedro
Processing Record 5 of Set 1 | avarua
Processing Record 6 of Set 1 | nome
Processing Record 7 of Set 1 | hermanus
Processing Record 8 of Set 1 | puerto ayora
Processing Record 9 of Set 1 | busselton
Processing Record 10 of Set 1 | punta arenas
Processing Record 11 of Set 1 | belozerskoye
Processing Record 12 of Set 1 | pasni
Processing Record 13 of Set 1 | port alfred
Processing Record 14 of Set 1 | atuona
Processing Record 15 of Set 1 | airai
Processing Record 16 of Set 1 | ilulissat
Processing Record 17 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 18 of Set 1 | alofi
Processing Record 19 of Set 1 | broome
Processing Record 20 of Set 1 | huarmey
Processing Record 21 of Set 1 | yulara
Processing Record 22 of Set 1 | carutapera
Processing Recor

Processing Record 41 of Set 4 | camapua
Processing Record 42 of Set 4 | huntsville
Processing Record 43 of Set 4 | wanning
Processing Record 44 of Set 4 | east london
Processing Record 45 of Set 4 | nizwa
Processing Record 46 of Set 4 | xining
Processing Record 47 of Set 4 | tasiilaq
Processing Record 48 of Set 4 | lata
Processing Record 49 of Set 4 | marawi
Processing Record 50 of Set 4 | uarini
Processing Record 1 of Set 5 | jiroft
City not found. Skipping...
Processing Record 2 of Set 5 | zaysan
Processing Record 3 of Set 5 | taoudenni
Processing Record 4 of Set 5 | tombouctou
Processing Record 5 of Set 5 | thompson
Processing Record 6 of Set 5 | asyut
Processing Record 7 of Set 5 | calamar
Processing Record 8 of Set 5 | farsund
Processing Record 9 of Set 5 | provideniya
Processing Record 10 of Set 5 | surt
Processing Record 11 of Set 5 | faanui
Processing Record 12 of Set 5 | ajdabiya
Processing Record 13 of Set 5 | laguna
Processing Record 14 of Set 5 | margate
Processing Record 1

Processing Record 31 of Set 8 | maniwaki
Processing Record 32 of Set 8 | aguimes
Processing Record 33 of Set 8 | sao raimundo das mangabeiras
Processing Record 34 of Set 8 | santa vitoria do palmar
Processing Record 35 of Set 8 | reykjavik
Processing Record 36 of Set 8 | lasa
Processing Record 37 of Set 8 | liwale
Processing Record 38 of Set 8 | ahipara
Processing Record 39 of Set 8 | dingle
Processing Record 40 of Set 8 | praia
Processing Record 41 of Set 8 | sibu
Processing Record 42 of Set 8 | nabire
Processing Record 43 of Set 8 | nokaneng
Processing Record 44 of Set 8 | klaksvik
Processing Record 45 of Set 8 | san quintin
Processing Record 46 of Set 8 | jibuti
Processing Record 47 of Set 8 | amderma
City not found. Skipping...
Processing Record 48 of Set 8 | mayo
Processing Record 49 of Set 8 | huron
Processing Record 50 of Set 8 | quthing
Processing Record 1 of Set 9 | montalban
Processing Record 2 of Set 9 | inderborskiy
City not found. Skipping...
Processing Record 3 of Set 9 |

Processing Record 21 of Set 12 | terra santa
Processing Record 22 of Set 12 | mehamn
Processing Record 23 of Set 12 | san isidro
Processing Record 24 of Set 12 | sorong
Processing Record 25 of Set 12 | sioux lookout
Processing Record 26 of Set 12 | yinchuan
Processing Record 27 of Set 12 | mizpe ramon
City not found. Skipping...
Processing Record 28 of Set 12 | forecariah
Processing Record 29 of Set 12 | grand centre
City not found. Skipping...
Processing Record 30 of Set 12 | penzance
Processing Record 31 of Set 12 | kailua
Processing Record 32 of Set 12 | bonthe
Processing Record 33 of Set 12 | hafizabad
Processing Record 34 of Set 12 | challans
Processing Record 35 of Set 12 | yar-sale
Processing Record 36 of Set 12 | yaan
Processing Record 37 of Set 12 | vilhena
Processing Record 38 of Set 12 | katherine
Processing Record 39 of Set 12 | flin flon
-----------------------------
Data Retrieval Complete      
-----------------------------


In [13]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pemangkat,1.1667,108.9667,81.75,73,4,1.57,ID,2022-05-31 16:52:19
1,Ust-Omchug,61.1500,149.6333,37.60,98,100,5.84,RU,2022-05-31 16:52:19
2,Bluff,-46.6000,168.3333,50.81,82,100,8.48,NZ,2022-05-31 16:50:29
3,San-Pedro,4.7485,-6.6363,81.23,74,28,7.90,CI,2022-05-31 16:52:19
4,Avarua,-21.2078,-159.7750,71.65,83,33,2.30,CK,2022-05-31 16:50:18
5,Nome,64.5011,-165.4064,52.16,66,0,0.00,US,2022-05-31 16:52:20
6,Hermanus,-34.4187,19.2345,52.81,69,53,13.20,ZA,2022-05-31 16:52:20
7,Puerto Ayora,-0.7393,-90.3518,80.56,81,98,13.20,EC,2022-05-31 16:50:34
8,Busselton,-33.6500,115.3333,52.02,78,51,3.80,AU,2022-05-31 16:52:21
9,Punta Arenas,-53.1500,-70.9167,42.91,70,0,9.22,CL,2022-05-31 16:50:22


In [14]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng",  "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]

In [16]:
city_data_df=city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Pemangkat,ID,2022-05-31 16:52:19,1.1667,108.9667,81.75,73,4,1.57
1,Ust-Omchug,RU,2022-05-31 16:52:19,61.1500,149.6333,37.60,98,100,5.84
2,Bluff,NZ,2022-05-31 16:50:29,-46.6000,168.3333,50.81,82,100,8.48
3,San-Pedro,CI,2022-05-31 16:52:19,4.7485,-6.6363,81.23,74,28,7.90
4,Avarua,CK,2022-05-31 16:50:18,-21.2078,-159.7750,71.65,83,33,2.30
5,Nome,US,2022-05-31 16:52:20,64.5011,-165.4064,52.16,66,0,0.00
6,Hermanus,ZA,2022-05-31 16:52:20,-34.4187,19.2345,52.81,69,53,13.20
7,Puerto Ayora,EC,2022-05-31 16:50:34,-0.7393,-90.3518,80.56,81,98,13.20
8,Busselton,AU,2022-05-31 16:52:21,-33.6500,115.3333,52.02,78,51,3.80
9,Punta Arenas,CL,2022-05-31 16:50:22,-53.1500,-70.9167,42.91,70,0,9.22


In [18]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")